In [ ]:
#Install the dependencies
!pip install git+https://github.com/tensorflow/cleverhans.git#egg=cleverhans
!pip install advertorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/tensorflow/cleverhans.git to /tmp/pip-install-fa47tsjr/cleverhans_82ea0f474ef048afbb86985d3d45f4e5
  Running command git clone -q https://github.com/tensorflow/cleverhans.git /tmp/pip-install-fa47tsjr/cleverhans_82ea0f474ef048afbb86985d3d45f4e5
  Attempting uninstall: cleverhans
    Found existing installation: cleverhans 3.0.1
    Uninstalling cleverhans-3.0.1:
      Successfully uninstalled cleverhans-3.0.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install git+https://github.com/CNOCycle/cleverhans.git@feature/tf2.x

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/CNOCycle/cleverhans.git (to revision feature/tf2.x) to /tmp/pip-req-build-1kwkjrys
  Running command git clone -q https://github.com/CNOCycle/cleverhans.git /tmp/pip-req-build-1kwkjrys
  Running command git checkout -b feature/tf2.x --track origin/feature/tf2.x
  Switched to a new branch 'feature/tf2.x'
  Branch 'feature/tf2.x' set up to track remote branch 'feature/tf2.x' from 'origin'.
  Created wheel for cleverhans: filename=cleverhans-3.0.1-py3-none-any.whl size=254394 sha256=323abcd00807e03caede951c83f821994b20e50b55e6f982853c401d42a2a140
  Stored in directory: /tmp/pip-ephem-wheel-cache-fd6hsm6r/wheels/7b/f7/6b/d754a65b70ab265387116e0a3b9c7bdb7adedc4e7afd04815f
Successfully built cleverhans
  Attempting uninstall: cleverhans
    Found existing installation: cleverhans 4.0.0
    Uninstalling cleverhans-4.0.0:
      Successfully uninstalled cleverhans-4.0.

In [ ]:
!pip install tensorflow==1.14

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 31 kB/s 
     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 488 kB 62.9 MB/s 
     |████████████████████████████████| 3.1 MB 45.5 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [ ]:
import tensorflow.compat.v1 as tf

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import warnings
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms

from cleverhans.compat import flags
from cleverhans.train import train
from cleverhans.dataset import MNIST
from cleverhans.utils import AccuracyReport
from cleverhans.utils_tf import model_eval

#Attack on PyTorch
from cleverhans.future.torch.attacks.fast_gradient_method import fast_gradient_method

#advertorch attacks
#from advertorch.attacks import CarliniWagnerL2Attack

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
# Creating a simple network
class LeNet5(torch.nn.Module):          
     
    def __init__(self):     
        super(LeNet5, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 6, 5, padding=2)
        self.conv2 = torch.nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)   
        self.fc2 = nn.Linear(120, 84)       
        self.fc3 = nn.Linear(84, 10)    
        
    def forward(self, x):
        x = F.relu(self.conv1(x))  
        x = F.max_pool2d(x, 2) 
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        
        return F.log_softmax(x,dim=-1)

In [ ]:
FLAGS = flags.FLAGS
NB_EPOCHS = 2
BATCH_SIZE = 128
LEARNING_RATE = .001

#Training the Network
def trainTorch(torch_model, train_loader, test_loader,
        nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE, train_end=-1, test_end=-1, learning_rate=LEARNING_RATE, optimizer=None):

    train_loss = []
    total = 0
    correct = 0
    step = 0
    for _epoch in range(nb_epochs):
      for xs, ys in train_loader:
        xs, ys = Variable(xs), Variable(ys)
        if torch.cuda.is_available():
          xs, ys = xs.cuda(), ys.cuda()
        optimizer.zero_grad()
        preds = torch_model(xs)
        # print("HI")
        loss = F.nll_loss(preds, ys)
        # print("HADSFSDF")
        loss.backward()  # calc gradients
        train_loss.append(loss.data.item())
        optimizer.step()  # update gradients

        preds_np = preds.cpu().detach().numpy()
        correct += (np.argmax(preds_np, axis=1) == ys.cpu().detach().numpy()).sum()
        total += train_loader.batch_size
        step += 1
        if total % 1000 == 0:
          acc = float(correct) / total
          print('[%s] Training accuracy: %.2f%%' % (step, acc * 100))
          total = 0
          correct = 0

In [ ]:
#Evaluate results on clean data
def evalClean(model1=None, test_loader=None):
    print("Evaluating single model results on clean data")
    total = 0
    correct = 0
    with torch.no_grad():
      model1.eval()
      for xs, ys in test_loader:
        xs, ys = Variable(xs), Variable(ys)
        if torch.cuda.is_available():
          xs, ys = xs.cuda(), ys.cuda()
        preds1 = model1(xs)
        preds_np1 = preds1.cpu().detach().numpy()
        finalPred = np.argmax(preds_np1, axis=1)
        correct += (finalPred == ys.cpu().detach().numpy()).sum()
        total += len(xs)
    acc = float(correct) / total
    print('Clean accuracy: %.2f%%' % (acc * 100))

#Evaluate results on adversarially perturbed 
def evalAdvAttack(fgsm_model=None, test_loader=None):
    print("Evaluating single model results on adv data")
    total = 0
    correct = 0
    fgsm_model.eval()
    for xs, ys in test_loader:
      if torch.cuda.is_available():
        xs, ys = xs.cuda(), ys.cuda()
      #pytorch fast gradient method
      xs = fast_gradient_method(fgsm_model, xs, eps=0.3, norm=np.inf, clip_min=0., clip_max=1.)
      # xs = fast_gradient_method(fgsm_model, xs, eps=0.1, norm=np.inf)
      xs, ys = Variable(xs), Variable(ys)
      preds1 = fgsm_model(xs)
      preds_np1 = preds1.cpu().detach().numpy()
      finalPred = np.argmax(preds_np1, axis=1)
      correct += (finalPred == ys.cpu().detach().numpy()).sum()
      total += test_loader.batch_size
    acc = float(correct) / total
    print('Adv accuracy: {:.3f}％'.format(acc * 100))


In [ ]:
#Adversarial Training
def advTrain(torch_model, train_loader, test_loader,
        nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE, train_end=-1, test_end=-1, learning_rate=LEARNING_RATE):
    optimizer = optim.Adam(torch_model.parameters(), lr=learning_rate)
    train_loss = []
    total = 0
    correct = 0
    totalAdv = 0
    correctAdv = 0
    step = 0
    # breakstep = 0
    for _epoch in range(nb_epochs):
      for xs, ys in train_loader:
        #Normal Training
        xs, ys = Variable(xs), Variable(ys)
        if torch.cuda.is_available():
          xs, ys = xs.cuda(), ys.cuda()
        optimizer.zero_grad()
        preds = torch_model(xs)
        loss = F.nll_loss(preds, ys)
        loss.backward()  # calc gradients
        train_loss.append(loss.data.item())
        optimizer.step()  # update gradients
        preds_np = preds.cpu().detach().numpy()
        correct += (np.argmax(preds_np, axis=1) == ys.cpu().detach().numpy()).sum()
        total += train_loader.batch_size

        #Adversarial Training
        xs = fast_gradient_method(torch_model, xs, eps=0.3, norm=np.inf, clip_min=0., clip_max=1.)
        xs, ys = Variable(xs), Variable(ys)
        if torch.cuda.is_available():
            xs, ys = xs.cuda(), ys.cuda()
        optimizer.zero_grad()
        preds = torch_model(xs)
        loss = F.nll_loss(preds, ys)
        loss.backward()  # calc gradients
        train_loss.append(loss.data.item())
        optimizer.step()  # update gradients
        preds_np = preds.cpu().detach().numpy()
        correctAdv += (np.argmax(preds_np, axis=1) == ys.cpu().detach().numpy()).sum()
        totalAdv += train_loader.batch_size
        
        step += 1
        if total % 1000 == 0:
          acc = float(correct) / total
          print('[%s] Clean Training accuracy: %.2f%%' % (step, acc * 100))
          total = 0
          correct = 0
          accAdv = float(correctAdv) / totalAdv
          print('[%s] Adv Training accuracy: %.2f%%' % (step, accAdv * 100))
          totalAdv = 0
          correctAdv = 0


In [ ]:
#Initialize model and data loader
model1 = LeNet5()
if torch.cuda.is_available():
  model1 = model1.cuda()
nb_epochs = 4
batch_size = 128
learning_rate = 0.001
train_end = -1
test_end = -1
report = AccuracyReport
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=True, download=True,
                    transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size)


In [ ]:
#Training the model
print("Training Model")
optimizer = optim.Adam(model1.parameters(), lr=learning_rate)
trainTorch(model1, train_loader, test_loader, nb_epochs, batch_size, train_end, test_end, learning_rate, optimizer = optimizer)

Training Model
[125] Training accuracy: 72.82%
[250] Training accuracy: 92.38%
[375] Training accuracy: 94.76%
[500] Training accuracy: 96.05%
[625] Training accuracy: 96.36%
[750] Training accuracy: 97.14%
[875] Training accuracy: 97.08%
[1000] Training accuracy: 97.27%
[1125] Training accuracy: 97.84%
[1250] Training accuracy: 97.79%
[1375] Training accuracy: 97.99%
[1500] Training accuracy: 98.00%
[1625] Training accuracy: 98.24%
[1750] Training accuracy: 98.31%
[1875] Training accuracy: 98.36%


In [ ]:
#Evaluation
evalClean(model1, test_loader)
evalAdvAttack(model1, test_loader)

Evaluating single model results on clean data
Clean accuracy: 97.98%
Evaluating single model results on adv data
Adv accuracy: 2.957％


In [ ]:
print("Training on Adversarial Samples")
advTrain(model1, train_loader, test_loader, nb_epochs, batch_size, train_end, test_end, learning_rate)

Training on Adversarial Samples
[125] Clean Training accuracy: 97.04%
[125] Adv Training accuracy: 35.41%
[250] Clean Training accuracy: 97.59%
[250] Adv Training accuracy: 58.71%
[375] Clean Training accuracy: 97.58%
[375] Adv Training accuracy: 67.29%
[500] Clean Training accuracy: 97.96%
[500] Adv Training accuracy: 75.28%
[625] Clean Training accuracy: 98.19%
[625] Adv Training accuracy: 82.05%
[750] Clean Training accuracy: 98.10%
[750] Adv Training accuracy: 84.31%
[875] Clean Training accuracy: 98.28%
[875] Adv Training accuracy: 86.57%
[1000] Clean Training accuracy: 98.15%
[1000] Adv Training accuracy: 86.32%
[1125] Clean Training accuracy: 98.58%
[1125] Adv Training accuracy: 87.50%
[1250] Clean Training accuracy: 98.70%
[1250] Adv Training accuracy: 88.00%
[1375] Clean Training accuracy: 98.56%
[1375] Adv Training accuracy: 88.52%
[1500] Clean Training accuracy: 98.48%
[1500] Adv Training accuracy: 89.19%
[1625] Clean Training accuracy: 98.65%
[1625] Adv Training accuracy: 8